# Comparing different search methods for Wikipedia

This Colab notebook accompanies this blog post: [Using LLMs for Search with Dense Retrieval and Reranking](https://txt.cohere.com/using-llms-for-search/).
In this notebook we study several different ways to query a Wikipedia database, including:
- Keyword search
- Dense retrieval
- Reranking
Furthermore, we combine the power of search with Cohere's Generate endpoint in order to output accurate answers in sentence format to a query.

# Imports

In [7]:
!pip install cohere weaviate-client

In [8]:
import weaviate
import cohere

# Add your Cohere API key here
# You can obtain a key by signing up in https://dashboard.cohere.com/ or https://docs.cohere.com/reference/key
cohere_api_key = ''

co = cohere.Client(cohere_api_key)

# Connect to the Weaviate demo databse containing 10M wikipedia vectors
# This uses a public READ-ONLY Weaviate API key
auth_config = weaviate.auth.AuthApiKey(api_key="76320a90-53d8-42bc-b41d-678647c6672e")
client = weaviate.Client(
    url="https://cohere-demo.weaviate.network/",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": cohere_api_key,
    }
)

client.is_ready() #check if True

True

# Keyword Search

We'll search for two queries using keyword search.
- Simple query: Who discovered penicillin?" (Answer: Alexander Fleming)
- Hard query: Who was the first person to win two Nobel prizes? (Answer: Marie Curie)

You will notice that keyword search performs very well with the simple query, and not so well with the hard one.

In [9]:
# This function performs keyword search
def keyword_search(query, results_lang='en', num_results=10):
  properties = ["text", "title", "url", "views", "lang", "_additional {distance}"]

  where_filter = {
  "path": ["lang"],
  "operator": "Equal",
  "valueString": results_lang
  }

  response = (
      client.query.get("Articles", properties)
      .with_bm25(
          query=query
      )
      .with_where(where_filter)
      .with_limit(num_results)
      .do()
  )
  result = response['data']['Get']['Articles']
  return result

In [10]:
# This function prints the results nicely
def print_result(result):
    """ Print results with colorful formatting """
    for item in result:
      print(f"\033[95m{item['title']} ({item['views']}) \033[0m")
      print(f"\033[4m{item['url']}\033[0m")
      print(item['text'])
      print()

In [11]:
simple_query = "Who discovered penicillin?"
keyword_search_results_simple = keyword_search(simple_query)
print_result(keyword_search_results_simple)

Penicillin (2000) 
https://en.wikipedia.org/wiki?curid=23312
When Alexander Fleming discovered the crude penicillin in 1928, one important observation he made was that many bacteria were not affected by penicillin. This phenomenon was realised by Ernst Chain and Edward Abraham while trying to identify the exact of penicillin. In 1940, they discovered that unsusceptible bacteria like "Escherichia coli" produced specific enzymes that can break down penicillin molecules, thus making them resistant to the antibiotic. They named the enzyme penicillinase. Penicillinase is now classified as member of enzymes called β-lactamases. These β-lactamases are naturally present in many other bacteria, and many bacteria produce them upon constant exposure to antibiotics. In most bacteria, resistance can be through three different mechanisms: reduced permeability in bacteria, reduced binding affinity of the penicillin-binding proteins (PBPs) or destruction of the antibiotic through the expression of β-l

In [12]:
hard_query = "Who was the first person to win two nobel prizes?"
keyword_search_results_hard = keyword_search(hard_query)
print_result(keyword_search_results_hard)

Neutrino (2000) 
https://en.wikipedia.org/wiki?curid=21485
In the 1960s, the now-famous Homestake experiment made the first measurement of the flux of electron neutrinos arriving from the core of the Sun and found a value that was between one third and one half the number predicted by the Standard Solar Model. This discrepancy, which became known as the solar neutrino problem, remained unresolved for some thirty years, while possible problems with both the experiment and the solar model were investigated, but none could be found. Eventually, it was realized that both were actually correct and that the discrepancy between them was due to neutrinos being more complex than was previously assumed. It was postulated that the three neutrinos had nonzero and slightly different masses, and could therefore oscillate into undetectable flavors on their flight to the Earth. This hypothesis was investigated by a new series of experiments, thereby opening a new major field of research that still con

# Dense Retrieval

Now we will use dense retrieval to search the answers for the two queries. Now you will notice that the results are good for both queries.

In [13]:
# This function performs dense retrieval
def dense_retrieval(query, results_lang='en', num_results=10):
    """
    Query the vectors database and return the top results.


    Parameters
    ----------
        query: str
            The search query

        results_lang: str (optional)
            Retrieve results only in the specified language.
            The demo dataset has those languages:
            en, de, fr, es, it, ja, ar, zh, ko, hi

    """

    nearText = {"concepts": [query]}
    properties = ["text", "title", "url", "views", "lang", "_additional {distance}"]
    # To filter by language
    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }
    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText)
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    result = response['data']['Get']['Articles']

    return result

In [14]:
simple_query = "Who discovered penicillin?"

dense_retrieval_results_simple = dense_retrieval(simple_query)
print_result(dense_retrieval_results_simple)

Alexander Fleming (2000) 
https://en.wikipedia.org/wiki?curid=1937
Sir Alexander Fleming (6 August 1881 – 11 March 1955) was a Scottish physician and microbiologist, best known for discovering the world's first broadly effective antibiotic substance, which he named penicillin. His discovery in 1928 of what was later named benzylpenicillin (or penicillin G) from the mould "Penicillium rubens" is described as the "single greatest victory ever achieved over disease." For this discovery, he shared the Nobel Prize in Physiology or Medicine in 1945 with Howard Florey and Ernst Boris Chain.

Penicillin (2000) 
https://en.wikipedia.org/wiki?curid=23312
Penicillin was discovered in 1928 by Scottish scientist Alexander Fleming as a crude extract of "P. rubens". Fleming's student Cecil George Paine was the first to successfully use penicillin to treat eye infection (Ophthalmia neonatorum) in 1930. The purified compound (penicillin F) was isolated in 1940 by a research team led by Howard Florey an

In [15]:
hard_query = "Who was the first person to win two Nobel prizes?"
dense_retrieval_results_hard = dense_retrieval(hard_query)
print_result(dense_retrieval_results_hard)

Nobel Prize (2000) 
https://en.wikipedia.org/wiki?curid=21201
Five people have received two Nobel Prizes. Marie Curie received the Physics Prize in 1903 for her work on radioactivity and the Chemistry Prize in 1911 for the isolation of pure radium, making her the only person to be awarded a Nobel Prize in two different sciences. Linus Pauling was awarded the 1954 Chemistry Prize for his research into the chemical bond and its application to the structure of complex substances. Pauling was also awarded the Peace Prize in 1962 for his activism against nuclear weapons, making him the only laureate of two unshared prizes. John Bardeen received the Physics Prize twice: in 1956 for the invention of the transistor and in 1972 for the theory of superconductivity. Frederick Sanger received the prize twice in Chemistry: in 1958 for determining the structure of the insulin molecule and in 1980 for inventing a method of determining base sequences in DNA. Karl Barry Sharpless was awarded the 2001 C

# ReRank

Rerank is a powerful method that will enhance any search model. In short, rerank takes a query and a set of responses (or documents), and will surface the ones that are the most relevant as answers to the query. We'll use Rerank to improve keyword search with the hard query.

In [29]:
def rerank_responses(query, responses, num_responses=10):
  reranked_responses = co.rerank(
    model = 'rerank-english-v2.0',
    query = query,
    documents = responses,
    top_n = num_responses,
  )
  return reranked_responses

In [39]:
hard_query = "Who was the first person to win two nobel prizes?"
keyword_searches_to_improve = keyword_search(hard_query, num_results = 100)

In [40]:
for r in keyword_searches_to_improve[:20]:
  print(r['title'], ':', r['text'])

Neutrino : In the 1960s, the now-famous Homestake experiment made the first measurement of the flux of electron neutrinos arriving from the core of the Sun and found a value that was between one third and one half the number predicted by the Standard Solar Model. This discrepancy, which became known as the solar neutrino problem, remained unresolved for some thirty years, while possible problems with both the experiment and the solar model were investigated, but none could be found. Eventually, it was realized that both were actually correct and that the discrepancy between them was due to neutrinos being more complex than was previously assumed. It was postulated that the three neutrinos had nonzero and slightly different masses, and could therefore oscillate into undetectable flavors on their flight to the Earth. This hypothesis was investigated by a new series of experiments, thereby opening a new major field of research that still continues. Eventual confirmation of the phenomenon 

In [41]:
reranked_keyword_responses = rerank_responses(hard_query, keyword_searches_to_improve, num_responses=100)

In [42]:
for r in reranked_keyword_responses[:10]:
  print("Title:", r.document['title'])
  print("URL:", r.document['url'])
  print("Text:", r.document['text'])
  print("Score:",  r.relevance_score)
  print()

Title: Nobel Prize
URL: https://en.wikipedia.org/wiki?curid=21201
Text: Five people have received two Nobel Prizes. Marie Curie received the Physics Prize in 1903 for her work on radioactivity and the Chemistry Prize in 1911 for the isolation of pure radium, making her the only person to be awarded a Nobel Prize in two different sciences. Linus Pauling was awarded the 1954 Chemistry Prize for his research into the chemical bond and its application to the structure of complex substances. Pauling was also awarded the Peace Prize in 1962 for his activism against nuclear weapons, making him the only laureate of two unshared prizes. John Bardeen received the Physics Prize twice: in 1956 for the invention of the transistor and in 1972 for the theory of superconductivity. Frederick Sanger received the prize twice in Chemistry: in 1958 for determining the structure of the insulin molecule and in 1980 for inventing a method of determining base sequences in DNA. Karl Barry Sharpless was awarded 

# Generating responses

Generative models are great at talking, but when it comes to answer questions with facts, they are prone to hallucinations. In other words, they can answer with the wrong answer. To prevent this, we first search for the documents that are relevant to the query (using dense retrieval, but we can use any method). We then feed them to the generative model, and instruct it to answer the question from the information from those documents.

The query is "How many people have won more than one Nobel prize?". You will notice that the model generates wrong answers, but when combined with search, it'll generate the correct answers.

In [48]:
import cohere
co = cohere.Client(cohere_api_key)

In [133]:
query = "How many people have won more than one Nobel prize?"

In [134]:
prediction_without_search = co.generate(
    prompt=query,
    max_tokens=50,
    num_generations=5,
    )

In [135]:
for p in prediction_without_search:
  print(p.text)

 There have been 24 Nobel Prize recipients who have won the prize more than once. Among them, John Bardeen has won the most Nobel Prizes, with four Nobel Prizes in Physics. Marie Curie has won the most Nobel Prizes among women, with the
 There have been 25 Nobel Prize recipients who have won more than one Nobel Prize. Of these, 14 are men and 11 are women. The most Nobel Prizes won by a single person is four, which was achieved by Marie Curie, who won the
 There are currently 18 individuals who have received more than one Nobel Prize. Of those, seven are still living. 
 There have been 24 Nobel Laureates who have won the Nobel Prize more than once.
 There are currently 13 people who have won more than one Nobel Prize. The most Nobel Prizes won by a single person is four, which is the case for Marie Curie. She won the Nobel Prize in Physics in 1903, in 1911, and in


In [142]:
responses = dense_retrieval(query, num_results = 20)
print_result(responses)

Nobel Peace Prize (2000) 
https://en.wikipedia.org/wiki?curid=26230922
, the Peace Prize has been awarded to 110 individuals and 27 organizations. 18 women have won the Nobel Peace Prize, more than any other Nobel Prize. Only two recipients have won multiple Prizes: the International Committee of the Red Cross has won three times (1917, 1944, and 1963) and the Office of the United Nations High Commissioner for Refugees has won twice (1954 and 1981). Lê Đức Thọ is the only person who refused to accept the Nobel Peace Prize.

Nobel Prize (2000) 
https://en.wikipedia.org/wiki?curid=21201
The strict rule against awarding a prize to more than three people is also controversial. When a prize is awarded to recognise an achievement by a team of more than three collaborators, one or more will miss out. For example, in 2002, the prize was awarded to Koichi Tanaka and John Fenn for the development of mass spectrometry in protein chemistry, an award that did not recognise the achievements of Franz

In [143]:
context = [r['text'] for r in responses]
context[:10]

[', the Peace Prize has been awarded to 110 individuals and 27 organizations. 18 women have won the Nobel Peace Prize, more than any other Nobel Prize. Only two recipients have won multiple Prizes: the International Committee of the Red Cross has won three times (1917, 1944, and 1963) and the Office of the United Nations High Commissioner for Refugees has won twice (1954 and 1981). Lê Đức Thọ is the only person who refused to accept the Nobel Peace Prize.',
 'The strict rule against awarding a prize to more than three people is also controversial. When a prize is awarded to recognise an achievement by a team of more than three collaborators, one or more will miss out. For example, in 2002, the prize was awarded to Koichi Tanaka and John Fenn for the development of mass spectrometry in protein chemistry, an award that did not recognise the achievements of Franz Hillenkamp and Michael Karas of the Institute for Physical and Theoretical Chemistry at the University of Frankfurt.',
 'The pr

In [144]:
prompt = f"""
Use the information provided below to answer the questions at the end. If the answer to the question is not contained in the provided information, say "The answer is not in the context".
---
Context information:
{context}
---
Question: How many people have won more than one Nobel prize?
"""

In [145]:
prediction_with_search = co.generate(
    prompt=prompt,
    num_generations=5,
    max_tokens=50,
    )

In [140]:
for p in prediction_with_search:
  print(p.text)

 The answer is Five people have received two Nobel Prizes.
 The answer is Five people have received two Nobel Prizes. Marie Curie received the Physics Prize in 1903 for her work on radioactivity and the Chemistry Prize in 1911 for the isolation of pure radium, making her the only person to be awarded a Nobel
 The answer is Five people have received two Nobel Prizes.
 The answer is The Curie family has received the most prizes, with four prizes awarded to five individual laureates. Marie Curie received the prizes in Physics (in 1903) and Chemistry (in 1911). Her husband, Pierre Curie, shared the
 The answer is Five people have received two Nobel Prizes. Marie Curie received the Physics Prize in 1903 for her work on radioactivity and the Chemistry Prize in 1911 for the isolation of pure radium, making her the only person to be awarded a Nobel
